# MATH 472 Homework 3
## Holly Bossart
## Due 02/06/2020

In [2]:
import pandas as pd
import numpy as np

## Problem 1
Implement iteratively reweighted least squares IRLS algorithm to reproduce the result in Example 2.5 of Computational Statistics book. 

(a) Print your result in the format of table 2.1 ousing starting value = (0.95913, 0).

In [4]:
# import the data
pd.read_csv('facerecognition.csv', delimiter = ' ')

,match,eyediff,nosecheekdiff,variabilityratio
0,1,0.009683,0.027914,1.086204
1,1,0.027614,0.017821,0.989840
2,1,0.015367,0.025832,1.010983
3,1,0.013371,0.024048,1.012724
4,1,0.011428,0.017801,1.044396
...,...,...,...,...
1037,1,0.014945,0.025952,0.946846
1038,0,0.179608,0.281835,0.968686
1039,1,0.048259,0.047284,0.988775
1040,0,0.014961,0.091083,0.976548
